# Find: opvragen van data

In dit notebook gaan verder we in op de MongoDB `find`-opdrachten (`find_one` en `find`), voor het opvragen van gegevens.

We beginnen met de standaardacties voor het importeren van de belangrijkste libraries, om verbinding te maken met de database, en om de voorbeeld-data in te lezen.

## Import en initialisatie

Dit is het vaste begin van elk notebook in deze reeks, zie de uitleg in [Connect](Connect.ipynb).

**Let op!** Het resultaat van de laatste opdracht (mongoimport) moet 0 zijn, anders is er sprake van een probleem. Dit moet je eerst (laten) oplossen: doorgaan met de rest heeft dan geen zin.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

print('Mongo version', pymongo.__version__)

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

client = pymongo.MongoClient('localhost', 27017)
db = client[dbname]
collection = db.contacts

mongopathfile = !cat mongopath
mongopath = mongopathfile[0]
print(mongopath)

collection.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c contacts adressen.json')

## Query: alle elementen

Als controle op de database zoeken we alle elementen.
(Voor een realistische database is dat ondoenlijk.)

In [ ]:
cursor = collection.find()
for obj in cursor:
    print(obj["name"])
    print(obj)

## Query: zoeken op naam

De volgende zoekopdracht is naar de gegevens van een of meerdere personen,
op basis van een deel van de gegevens.
Je geeft in de zoekopdracht een *gedeeltelijk ingevuld document*: het query-document.
de zoekopdracht vindt dan alle documenten die met die invulling overeenkomen.

> Opmerking: dit komt overeen met het `where`-deel in een SQL-query.

In [ ]:
cursor = collection.find({"name": "Anna Verschuur"})
for obj in cursor:
    print(obj)

Als het aantal resultaten klein is, dan is het vaak handiger om de cursor direct om te zetten in een `list`.
We hebben dan niet steeds een for-opdracht nodig om de elementen af te drukken.

**Let op** na de opdracht ``list(cursor)`` (of na de `for`-opdracht) het is de cursor "leeg"; ga na wat er gebeurt als je deze opdracht tweemaal achter elkaar uitvoert.

In [ ]:
cursor = collection.find({"name": "Anna Verschuur"})
list(cursor)

Je kunt zoekwaarde(n) gebruiken die willekeurig diep in het document genest zijn, bijvoorbeeld:

In [ ]:
list(collection.find({"address.city": "Amsterdam"}))

**Opdracht** maak hieronder een query voor het zoeken van alle inwoners van Rotterdam in de database.

## Find: reguliere expressies

Met behulp van reguliere expressies kun je zoeken op waarden die voor een deel bepaald zijn. In het voorbeeld zoeken we op namen waar `schuur` in voorkomt; we maken daarbij geen verschil tussen hoofd- en kleine letters (`re.IGNORECASE`).

> MongoDB/pymongo gebruikt hiervoor de Python reguliere expressies, zie: https://docs.python.org/3/library/re.html en https://docs.python.org/3/howto/regex.html.

> In het geval van SQL gebruik je hiervoor de LIKE constructie met wildcards ("jokers").

In [ ]:
list(collection.find({"name": re.compile("schuur", flags=re.IGNORECASE)}))

**Opdracht** Zoek personen die in Amsterdam of in Rotterdam wonen (door een handige reguliere expressie te kiezen).

## Zoeken naar één document

Voor het zoeken naar één enkel document gebruik je: `find_one`.
Als er geen document gevonden wordt is het resultaat `None`.
(Dit is *de lege waarde* in Python.)

In [ ]:
anna = collection.find_one({"name": "Anna Verschuur"})
anna

## Samengestelde queries

Als je meerdere eigenschappen gebruikt in een query, dan heeft dit de betekenis van een **and**:
de gezochte documenten voldoen aan alle deel-eisen.

In [ ]:
cursor = collection.find({"name": "Anna Verschuur", "address.city": "Amsterdam"})
for obj in cursor:
    print(obj)

**Opdracht** maak een zoekopdracht voor alle "Anna"s die in Amsterdam wonen.

Soms heb je andere samenstellingen nodig, bijvoorbeeld als je zoekt naar personen in Amsterdam of in Rotterdam.
In dit geval gebruik je een `$or` met een lijst van deel-eisen.
(Je hebben eerder dit probleem opgelost met een reguliere expressie.)

In [ ]:
cursor = collection.find(
    {"$or": [{"address.city": "Rotterdam"}, {"address.city": "Amsterdam"}]})
list(cursor)

**Opdracht** zoek alle "Anna"s die in Amsterdam of in Rotterdam wonen.

## Projectie

In de voorbeelden hierboven hebben we de complete documenten laten zien.
Vaak ben je maar in een paar onderdelen geïnteresseerd.
In een *projectie* geef je de gewenste onderdelen van het resultaat aan.
Zo'n projectie bevat de velden van het document die je wilt zien, met een "1" als waarde.
Je kunt ook aangeven welke velden je wilt weglaten (exclusie): die geef je aan met een "0" als waarde.
Dit gebruiken we bijvoorbeeld als we de *key* (`_id`) niet nodig hebben.

> Opmerking: projectie komt overeen met het SELECT-deel in een SQL-query

> ***NB: hier voeren we teveel nieuwe dingen tegelijk in - AANPASSEN***

In [ ]:
query = {"address.city": "Amsterdam"}
projection = {"_id":0, "name": 1, "address": 1}
list1 = list(collection.find(query, projection))
list1

### Weergave als tabel
Je kunt een dergelijke lijst ook als (pandas) tabel weergeven.
Daarbij kun je ook de volgorde van de elementen (kolommen) aangeven.
Let hierbij op de dubbele `[[ ]]`: dit is ook een *projectie*, maar nu in Python/pandas.

> Het is niet netjes om het geneste document "address" zo te laten staan, maar dat laten we nu even voor wat het is.

In [ ]:
df0 = pd.DataFrame(list1)
display(df0[["name", "address"]])

## Sorteren

Met behulp van `sort` pas je de volgorde van de documenten in het resultaat aan.
Je geeft daarbij het veld en de sorteerrichting aan.

In [ ]:
cursor = collection.find({"address.city": "Amsterdam"}, {"_id":0, "name": 1, "address": 1})
cursor.sort("address.street", pymongo.DESCENDING)
df0 = pd.DataFrame(list(cursor))
display(df0[["name", "address"]])

## Alleen documenten met een bepaald veld

In plaats van de *waarde* van een veld, kunnen we ook het predicaat-waarde `{"$exists": True}` opgeven: we vinden dan alleen die documenten die een waarde hebben voor dit veld.
Dit is in MongoDB een zinvolle query, omdat documenten verschillend kunnen zijn van structuur.

> In SQL heeft een rij altijd alle velden, maar een veld kan wel *leeg* (NULL) zijn. Dat komt het meest in de buurt van deze vorm.

In [ ]:
projection = {"_id":0, "name": 1, "email": 1, "tel": 1}
cursor = collection.find({"email": {"$exists": True}}, projection)
list(cursor)

## Documenten met een ontbrekend veld

Het tegenovergestelde is ook vaak nuttig: in welke documenten ontbreekt een bepaald veld? (Via: `"$exists": False`.)

In [ ]:
cursor = collection.find({"address": {"$exists": True}, "address.postcode": {"$exists": False}})
list(cursor)

**Opdracht** Maak een zoekopdracht voor alle documenten zonder telefoonnummer.

## Welke waarden in de database?

Soms wil je weten welke waarden voor een bepaald veld in de documenten in de database voorkomen.
Bijvoorbeeld: welke plaatsen komen voor in de database?
Met behulp van `distinct` kun je dit opvragen.

In [ ]:
collection.distinct("address.city")

**Opdracht** Welke namen komen in de database voor?

## Meervoudige waarden

Zoals je wellicht gezien hebt, kan een contact-document meerdere email-adressen bevatten.
Voor de `find`-opdrachten maakt dit niets uit:

1. in het resultaat zie je voor het email-veld soms een enkelvoudige waarde, en soms een array van waarden.
2. bij het zoeken naar een document met een bepaald email-adres kun je dezelfde vorm hanteren als voor een enkelvoudige waarde

Het eerste heb je al in de eerdere voorbeelden gezien (controleer dit eventueel).
Van dit laatste zie je hieronder een voorbeeld:

In [ ]:
projection = {"name":1, "address.city":1, "email":1}
list(collection.find({"email": "lhmdebruin@hotmail.com"}, projection))

**Opdracht**
Maak een zoekopdracht voor alle contacten die een hotmail-mailadres hebben.